# XGBoost和GBDT的执行速度的比较

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score

import xgboost as xgb

E:\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
mpl.rcParams['font.sans-serif'] = [u'SimHei']
mpl.rcParams['axes.unicode_minus'] = False

In [3]:
def notEmpty(s):
    return s != ''

In [7]:
names = ['CRIM','ZN', 'INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT']
path = "./boston_housing.data"
## 由于数据文件格式不统一，所以读取的时候，先按照一行一个字段属性读取数据，然后再按照每行数据进行处理
fd = pd.read_csv(path, header=None)
data = np.empty((len(fd), 14))
# data.shape
for i, d in enumerate(fd.values):
    d = map(float, filter(notEmpty, d[0].split(' ')))
    data[i] = list(d)

x, y = np.split(data, (13,), axis=1)
y = y.ravel()

print ("样本数据量:%d, 特征个数：%d" % x.shape)
print ("target样本数据量:%d" % y.shape[0])

样本数据量:506, 特征个数：13
target样本数据量:506


In [11]:
#查看数据信息
X_DF = pd.DataFrame(x, columns=names)
X_DF.info()
X_DF.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
CRIM       506 non-null float64
ZN         506 non-null float64
INDUS      506 non-null float64
CHAS       506 non-null float64
NOX        506 non-null float64
RM         506 non-null float64
AGE        506 non-null float64
DIS        506 non-null float64
RAD        506 non-null float64
TAX        506 non-null float64
PTRATIO    506 non-null float64
B          506 non-null float64
LSTAT      506 non-null float64
dtypes: float64(13)
memory usage: 51.5 KB


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000


In [15]:
#数据的分割，
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=14)
print ("训练数据集样本数目：%d, 测试数据集样本数目：%d" % (x_train.shape[0], x_test.shape[0]))

训练数据集样本数目：404, 测试数据集样本数目：102


E:\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [23]:
from time import *
from xgboost import XGBRegressor,XGBClassifier
t=time()
xgb=XGBRegressor(max_depth=3,n_estimators=100,learning_rate=0.1,reg_alpha=1,reg_lambda=1)
xgb.fit(x_train,y_train,eval_metric='rmse',eval_set=[(x_train,y_train),(x_test,y_test)],early_stopping_rounds=40)

xgb.n_estimators=xgb.best_iteration
y_pred=xgb.predict(x_test)
print('xgboost r2误差',r2_score(y_test,y_pred))
print('执行速度：',time()-t)

[12:10:01] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:22.0822	validation_1-rmse:19.6198
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 40 rounds.
[1]	validation_0-rmse:20.0041	validation_1-rmse:17.7532
[2]	validation_0-rmse:18.126	validation_1-rmse:16.0439
[3]	validation_0-rmse:16.4381	validation_1-rmse:14.5261
[4]	validation_0-rmse:14.9153	validation_1-rmse:13.1844
[5]	validation_0-rmse:13.5522	validation_1-rmse:11.9971
[6]	validation_0-rmse:12.3255	validation_1-rmse:10.9011
[7]	validation_0-rmse:11.2126	validation_1-rmse:9.91312
[8]	validation_0-rmse:10.2159	validation_1-rmse:9.05283
[9]	validation_0-rmse:9.32347	validation_1-rmse:8.28988
[10]	validation_0-rmse:8.52011	validation_1-rmse:7.61867
[11]	validation_0-rmse:7.80087	validation_1-rmse:7.02343
[12]

In [24]:
#GDBT
from sklearn.ensemble import GradientBoostingRegressor
t=time()
algo = GradientBoostingRegressor(n_estimators=10, learning_rate=1.0, max_depth=3)
algo.fit(x_train,y_train)
y1_pred=algo.predict(x_test)
print('GBDT的r2误差',algo.score(x_test,y_test))
print('执行速度：',time()-t)

GBDT的r2误差 0.792597017379592
执行速度： 0.011969566345214844
